In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
dataset = pd.read_csv('Video_Games_Sales_as_at_22_Dec_2016.csv')

In [4]:
dataset.drop(columns = ['Year_of_Release', 'Developer', 'Publisher', 'Platform'], inplace = True)

In [5]:
print('Feature name || Total missing values')
print(dataset.isna().sum())

Feature name || Total missing values
Name               2
Genre              2
NA_Sales           0
EU_Sales           0
JP_Sales           0
Other_Sales        0
Global_Sales       0
Critic_Score    8582
Critic_Count    8582
User_Score      9129
User_Count      9129
Rating          6769
dtype: int64


In [6]:
X = dataset.iloc[:, :].values
X = np.delete(X, 6, 1)
y = dataset.iloc[:, 6:7].values

In [7]:
print('Feature name || Total missing values')
print(dataset.isna().sum())

Feature name || Total missing values
Name               2
Genre              2
NA_Sales           0
EU_Sales           0
JP_Sales           0
Other_Sales        0
Global_Sales       0
Critic_Score    8582
Critic_Count    8582
User_Score      9129
User_Count      9129
Rating          6769
dtype: int64


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [9]:
games_in_training_set = X_train[:, 0]
games_in_test_set = X_test[:, 0]

In [10]:
X_train = X_train[:, 1:]
X_test = X_test[:, 1:]

In [11]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy = 'mean')
X_train[:, [5 ,6, 7, 8]] = imputer.fit_transform(X_train[:, [5, 6, 7, 8]])
X_test[:, [5 ,6, 7, 8]] = imputer.transform(X_test[:, [5, 6, 7, 8]])

In [12]:
pip install sklearn-pandas

Note: you may need to restart the kernel to use updated packages.


In [13]:
from sklearn.impute import SimpleImputer
categorical_imputer = SimpleImputer(strategy = 'constant', fill_value = 'NA')
X_train[:, [0, 9]] = categorical_imputer.fit_transform(X_train[:, [0, 9]])
X_test[:, [0, 9]] = categorical_imputer.transform(X_test[:, [0, 9]])

In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [0, 9])], remainder = 'passthrough') 
X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

In [15]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [16]:
from xgboost import XGBRegressor
model = XGBRegressor(n_estimators = 200, learning_rate= 0.08)
model.fit(X_train, y_train)

E:\others\anaconda\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.08, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=200, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [17]:
y_pred = model.predict(X_test)

In [18]:
games_in_test_set = games_in_test_set.reshape(-1, 1)
y_pred = y_pred.reshape(-1, 1)
predictions = np.concatenate([games_in_test_set, y_pred, y_test], axis = 1)
predictions = pd.DataFrame(predictions, columns = ['Name', 'Predicted_Global_Sales', 'Actual_Global_Sales'])

In [19]:
print(predictions)

                                Name Predicted_Global_Sales  \
0                       R-Type Final               0.186028   
1       The Terminator: Dawn of Fate                0.41634   
2        Dead to Rights: Retribution               0.293324   
3              Skylanders SWAP Force               2.169221   
4                               DiRT               1.161763   
...                              ...                    ...   
5011                Dynasty Warriors               0.457753   
5012  Transformers: Dark of the Moon               0.171426   
5013           Brunswick Pro Bowling               0.204492   
5014       Valentino Rossi: The Game               0.067381   
5015         Street Fighter X Tekken               0.193131   

     Actual_Global_Sales  
0                   0.19  
1                   0.41  
2                   0.28  
3                   2.15  
4                   1.05  
...                  ...  
5011                0.51  
5012                0.17  


In [20]:
from sklearn.metrics import r2_score, mean_squared_error
import math
r2_score = r2_score(y_test, y_pred)
rmse = math.sqrt(mean_squared_error(y_test, y_pred))

print(f"r2 score of the model : {r2_score:.3f}")
print(f"Root Mean Squared Error of the model : {rmse:.3f}")

r2 score of the model : 0.732
Root Mean Squared Error of the model : 0.743
